# Coursera Applied Data Science Capstone Week 3
This is the Notebook created for the course Applied Data Science

In [104]:
# Install packages
!pip install beautifulsoup4 -q
!pip install lxml -q

In [111]:
import numpy
import pandas
import requests
from bs4 import BeautifulSoup

In [112]:
# Download the HTML page
!wget -q -O 'wikipedia_canada_postal_codes.html' 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [113]:
# Define the function that will be used on groupby
def joinstr(x):
    return pandas.Series(dict(Neighbourhood = "%s" % ', '.join(x['Neighbourhood'])))

In [116]:
# DataFrame to hold the data columns
postal_codes = pandas.DataFrame(columns=['PostalCode', 'Borough', 'Neighbourhood'])

with open('wikipedia_canada_postal_codes.html') as html_file:
    # Load html
    soap = BeautifulSoup(html_file)
    
    # select the table
    postalcode_table = soap.table
    
    # create iterator
    rows = iter(postalcode_table.find_all('tr'))
    
    # skip the table head
    next(rows)
    
    # foreach row
    for postal_row in rows:
        # select each col
        postalcode = postal_row.contents[1].text.strip()
        borough = postal_row.contents[3].text.strip()
        neighbourhood = postal_row.contents[5].text.strip()
        
        # if not assigned borough: skip it
        if 'assigned' in borough:
            continue
        
        # if not assigned neighbourhood: copy the borough
        if  'assigned' in neighbourhood:
            neighbourhood = borough
        
        # append the row into the data frame
        postal_codes = postal_codes.append({  'PostalCode': postalcode, 'Borough': borough, 'Neighbourhood': neighbourhood}, ignore_index=True)

postal_codes = postal_codes.groupby(['PostalCode', 'Borough']).apply(joinstr).reset_index()
postal_codes

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [117]:
postal_codes.shape

(103, 3)